In [1]:
################################################
#Based on code provided in SI of Swainston 2016#
################################################

import re
import math
import cobra
import gurobipy


def manipulate_id(ID):
    #change sbml reaction IDs in cobra metabolite IDs
    #example: maniupulate_id("EX_ca2(e)")="ca2_e"
    ID=ID.strip(")")
    ID=re.sub("EX_",'',ID)
    ID=re.sub("\(","_",ID)
    #print(med)
    return(ID)


def find_exchanges(metabolite):
    #1 - retrieve metabolite object in cobra model
    #2 - iterate through reactions associated with metabolite
    #3 - find exchange rxn, return id if found or else return nothing
    met=model.metabolites.get_by_id(metabolite)
    count=0
    for r in met.reactions:
        ex=""
        ex=model.reactions.get_by_id(r.id)
        if r.boundary==True:
            #print( "%s\t%s\t%s" % (ex.id,ex.boundary,ex.reaction))
            return(ex.id)
            count+=1
        else:
            pass
    if count==0:
        print("No exchanges found")

def set_import_bounds(model, rxn_name_list, value):
    '''Sets the import bounds.'''
    #model = sbml.getModel()
    # convert single entries to lists
    if isinstance(rxn_name_list, str):
        rxn_name_list = [rxn_name_list]
    if isinstance(value, (int, float, complex)):
        value = [value] * len(rxn_name_list)
    for index, rID in enumerate(rxn_name_list):
        try:
            reaction= model.reactions.get_by_id(rID)
            val = abs(value[index])
            try:
                reaction.lower_bound=-val
            except:
                pass
                #print('reaction %s not import' % rID)
        except:
            pass
            #print('reaction %s not found' % rID)

def set_infinite_bounds(model):
    #set infinite bounds
    for r in model.reactions:
        r.lower_bound=-1e6
        r.upper_bound=1e6
def block_imports(model):
    #block import reactions
    for r in model.exchanges:
        #nR, nP = 0, 0
        model.reactions.get_by_id(r.id).lower_bound=0
def max_flux(model,normoxic,media,carbon_source):
    set_infinite_bounds(model)
    block_imports(model)
    if normoxic:
            model.reactions.get_by_id('EX_o2_LPAREN_e_RPAREN_').lower_bound=-1e6           
    set_import_bounds(model,carbon_source,1)
    set_import_bounds(model,new_media,-1e6) 
    model.optimize()
    #pfba=cobra.flux_analysis.pfba(model)
    #loopless=cobra.flux_analysis.loopless_solution(model)
    #print(loopless
    print(model.summary())
    #print(model.summary(pfba))

In [2]:
#read in cobra based model saved in sbml format
model = cobra.io.read_sbml_model("../GEMs/recon2_2.xml")

cobra/io/sbml.py:235 UserWarning: M_h_x appears as a reactant and product FAOXC220200x


In [3]:
new_media=[]
new_carbons=[]
new_carbons.append(find_exchanges("glc_D_e"))
not_found=[]
media = [
        'EX_ca2(e)',
        'EX_cl(e)',
        'EX_fe2(e)',
        'EX_fe3(e)',
        'EX_h(e)',
        'EX_h2o(e)',
        'EX_k(e)',
        'EX_na1(e)',
        'EX_nh4(e)',
        'EX_so4(e)',
        'EX_pi(e)'
    ]

#print("Media\n")

for med in media:
    med=med.strip(")")
    med=re.sub("EX_",'',med)
    med=re.sub("\(","_",med)
    #print(med)
    new_media.append(find_exchanges(med))

#print("\n")
#print("Carbon Sources\n")

for carbon_source in [
                # sugars
                'EX_glc(e)',
                'EX_fru(e)',
                # fatty acids
                'EX_ppa(e)',        # C3:0
                'EX_but(e)',        # C4:0
                'EX_hx(e)',         # C6:0
                'EX_octa(e)',       # C8:0
                'EX_HC02175(e)',    # C10:0
                'EX_HC02176(e)',    # C12:0
                'EX_ttdca(e)',      # C14:0
                'EX_hdca(e)',       # C16:0
                'EX_ocdca(e)',      # C18:0
                'EX_arach(e)',      # C20:0
                'EX_docosac_',      # C22:0
                'EX_lgnc(e)',       # C24:0
                'EX_hexc(e)',       # C26:0
                # amino acids
                'EX_ala_L(e)',
                'EX_arg_L(e)',
                'EX_asn_L(e)',
                'EX_asp_L(e)',
                'EX_cys_L(e)',
                'EX_gln_L(e)',
                'EX_glu_L(e)',
                'EX_gly(e)',
                'EX_his_L(e)',
                'EX_ile_L(e)',
                'EX_leu_L(e)',
                'EX_lys_L(e)',
                'EX_met_L(e)',
                'EX_phe_L(e)',
                'EX_pro_L(e)',
                'EX_ser_L(e)',
                'EX_thr_L(e)',
                'EX_trp_L(e)',
                'EX_tyr_L(e)',
                'EX_val_L(e)',
        ]:
    try:
        new_id=manipulate_id(carbon_source)
        new_carbons.append(find_exchanges(new_id))
    except:
        pass
        #print("%s\t%s" % (new_id,"Did not work with find_exchanges"))
        #not_found.append(new_id)
#print(new_carbons,"\n",not_found)


#not_found.remove("glc_e")
new_carbons.append(find_exchanges("dca_e"))
#not_found.remove('HC02175_e')
new_carbons.append(find_exchanges("ddca_e"))
#not_found.remove('HC02176_e')
new_carbons.append(find_exchanges('docosac_e'))
#not_found.remove('docosac_')

#print(new_carbons,not_found)

In [10]:
#objective = 'DM_atp_c_'

model.objective='DM_atp_c_'
#model.objective='r0173'
#model.objective='EX_atp_LPAREN_e_RPAREN_'
model.objective.direction='max'
model.solver='gurobi'
max_flux(model,True,new_media,new_carbons[0])

IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ------------
pi_e   1e+06     HC00250_e  1e+06
so4_e  1e+06     h2o2_e     1e+06
nh4_e  1e+06     no_e       1e+06
h_e    1.67e+05  h2o_e      8.33e+05
                 ppi_e      5e+05
                 o2s_e      3.33e+05
                 o2_e       1.59e-08
None


In [11]:
for normoxic in [True,False]:
    for carbon in new_carbons:
        #print(model.reactions.get_by_id(carbon).reaction)
        model.objective='DM_atp_c_'
        model.objective.direction='max'
        model.solver='gurobi'
        max_flux(model,normoxic,new_media,carbon)

IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ------------
pi_e   1e+06     HC00250_e  1e+06
so4_e  1e+06     h2o2_e     1e+06
nh4_e  1e+06     no_e       1e+06
h_e    1.67e+05  h2o_e      8.33e+05
                 ppi_e      5e+05
                 o2s_e      3.33e+05
                 o2_e       1.59e-08
None
IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ------------
pi_e   1e+06     HC00250_e  1e+06
so4_e  1e+06     h2o2_e     1e+06
nh4_e  1e+06     no_e       1e+06
h_e    1.67e+05  h2o_e      8.33e+05
                 ppi_e      5e+05
                 o2s_e      3.33e+05
                 o2_e       1.59e-08
None
IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ------------
pi_e   1e+06     HC00250_e  1e+06
so4_e  1e+06     h2o2_e     1e+06
nh4_e  1e+06     no_e       1e+06
h_e    1.67e+05  h2o_e      8.33e+05
                 ppi_e      5e+05
                 o2s_e 

IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ------------
pi_e   1e+06     HC00250_e  1e+06
so4_e  1e+06     h2o2_e     1e+06
nh4_e  1e+06     no_e       1e+06
h_e    1.67e+05  h2o_e      8.33e+05
                 ppi_e      5e+05
                 o2s_e      3.33e+05
                 o2_e       1.59e-08
None
IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ------------
pi_e   1e+06     HC00250_e  1e+06
so4_e  1e+06     h2o2_e     1e+06
nh4_e  1e+06     no_e       1e+06
h_e    1.67e+05  h2o_e      8.33e+05
                 ppi_e      5e+05
                 o2s_e      3.33e+05
                 o2_e       1.59e-08
None
IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ------------
pi_e   1e+06     HC00250_e  1e+06
so4_e  1e+06     h2o2_e     1e+06
nh4_e  1e+06     no_e       1e+06
h_e    1.67e+05  h2o_e      8.33e+05
                 ppi_e      5e+05
                 o2s_e 

IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ------------
pi_e   1e+06     HC00250_e  1e+06
so4_e  1e+06     h2o2_e     1e+06
nh4_e  1e+06     no_e       1e+06
h_e    1.67e+05  h2o_e      8.33e+05
                 ppi_e      5e+05
                 o2s_e      3.33e+05
                 o2_e       1.59e-08
None
IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ------------
pi_e   1e+06     HC00250_e  1e+06
so4_e  1e+06     h2o2_e     1e+06
nh4_e  1e+06     no_e       1e+06
h_e    1.67e+05  h2o_e      8.33e+05
                 ppi_e      5e+05
                 o2s_e      3.33e+05
                 o2_e       1.59e-08
None
IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ------------
pi_e   1e+06     HC00250_e  1e+06
so4_e  1e+06     h2o2_e     1e+06
nh4_e  1e+06     no_e       1e+06
h_e    1.67e+05  h2o_e      8.33e+05
                 ppi_e      5e+05
                 o2s_e 

In [5]:
test=cobra.io.read_sbml_model('../GEMs/0349f526-7816-4a7d-9967-1f75dd9ff00a.xml')
#test.objective='r0173'
#test.objective='EX_atp_LPAREN_e_RPAREN_'
test.add_reaction(model.reactions.DM_atp_c_)
test.objective=test.reactions.DM_atp_c_.id
test.objective.direction="max"
#test.objective.
test.solver='gurobi'
max_flux(test,True,new_media,new_carbons[0])

IN FLUXES          OUT FLUXES       OBJECTIVES
-----------------  ---------------  ----------------
glc_D_e  1         fru_e  1         DM_atp_c_  1e+06
h_e      1.63e-08  h2o_e  1.57e-08
None


In [6]:
for normoxic in [True,False]:
    if normoxic == False:
        pass
    for carbon in new_carbons:
        test=cobra.io.read_sbml_model('../GEMs/0349f526-7816-4a7d-9967-1f75dd9ff00a.xml')
        #test.objective='r0173'
        #test.objective='EX_atp_LPAREN_e_RPAREN_'
        test.add_reaction(model.reactions.DM_atp_c_)
        test.objective=test.reactions.DM_atp_c_.id
        test.objective.direction='max'
        test.solver='gurobi'
        try:
            print(test.reactions.get_by_id(carbon).reactants)
            max_flux(test,normoxic,new_media,carbon)
        except:
            print("Carbon not found", model.reactions.get_by_id(carbon).reactants)
        

[<Metabolite glc_D_e at 0x7f22deb70080>]
IN FLUXES          OUT FLUXES       OBJECTIVES
-----------------  ---------------  ----------------
glc_D_e  1         fru_e  1         DM_atp_c_  1e+06
h_e      1.63e-08  h2o_e  1.57e-08
None
[<Metabolite fru_e at 0x7f22d7d3ca90>]
IN FLUXES        OUT FLUXES       OBJECTIVES
---------------  ---------------  ----------------
o2_e   3.33e+05  h_e    4.44e+05  DM_atp_c_  1e+06
h2o_e  2.22e+05  o2s_e  4.44e+05
fru_e  1         oxa_e  2
                 co2_e  2
None
Carbon not found [<Metabolite ppa_e at 0x7f22e3e36358>]
Carbon not found [<Metabolite but_e at 0x7f22e40faa90>]
[<Metabolite hx_e at 0x7f22d196b048>]
IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  ----------------
o2_e   1.66e+05  o2s_e      2.22e+05  DM_atp_c_  1e+06
h2o_e  1.11e+05  h_e        2.22e+05
hx_e   1         acetone_e  2
                 pi_e       2.54e-08
                 oxa_e      1.63e-08
None
[<Metabolite octa_e at 0x7f22cffaa8

[<Metabolite ocdca_e at 0x7f228b096860>]
IN FLUXES       OUT FLUXES        OBJECTIVES
--------------  ----------------  ----------------
h2o_e    3      etoh_e     3      DM_atp_c_  1e+06
ocdca_e  1      acetone_e  0.667
h_e      0.333  ptdca_e    0.667
None
[<Metabolite arach_e at 0x7f229266eba8>]
IN FLUXES      OUT FLUXES    OBJECTIVES
-------------  ------------  ----------------
h_e  3.33e-08                DM_atp_c_  1e+06
None
Carbon not found [<Metabolite lgnc_e at 0x7f22e3ef6320>]
Carbon not found [<Metabolite hexc_e at 0x7f22e3f87828>]
[<Metabolite ala_L_e at 0x7f227af6a080>]
IN FLUXES        OUT FLUXES       OBJECTIVES
---------------  ---------------  ----------------
h2o_e  3.41e-08  pi_e   1.83e-08  DM_atp_c_  1e+06
                 adn_e  1.82e-08
                 h_e    1.7e-08
None
[<Metabolite arg_L_e at 0x7f2278463ac8>]
IN FLUXES    OUT FLUXES       OBJECTIVES
-----------  ---------------  ----------------
             h2o_e  1.84e-08  DM_atp_c_  1e+06
None
[<Metaboli

In [ ]:
test=cobra.io.read_sbml_model('../GEMs/0349f526-7816-4a7d-9967-1f75dd9ff00a.xml')
#test.objective='r0173'
test.objective='EX_atp_LPAREN_e_RPAREN_'
test.objective.direction='max'
test.solver='gurobi'
max_flux(test,True,new_media,new_carbons[0])